In [11]:
from __future__ import annotations

import os
import re
import math
import json
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Iterable, Literal
import requests
import numpy as np
import pandas as pd
from pandas import json_normalize
from bs4 import BeautifulSoup
from zeep import Client, helpers
import xmltodict
from utils import safe_serialize, listar_ops
from tqdm import tqdm

In [2]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [3]:
periodos = ['2022-2026'] # <- Elegir períodos

In [27]:
def get_boletin(boletin_num):
    """
    Consulta la API del Senado y devuelve un DataFrame con:
    boletin, materias, titulo, camara_origen, fecha_ingreso, ley, estado
    """
    
    try:
        url = f"https://tramitacion.senado.cl/wspublico/tramitacion.php?boletin={boletin_num}"
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # lanza error si la respuesta es inválida
        soup = BeautifulSoup(response.content, "xml")

        materias = [m.text.strip() for m in soup.find_all("materia")]
        titulo = soup.find("titulo")
        cam_origen = soup.find("camara_origen")
        fecha_origen = soup.find("fecha_ingreso")
        ley = soup.find("leynro")
        estado = soup.find("estado")

        data = {
            "boletin": boletin_num,
            "materias": materias,
            "materias_texto": "; ".join(materias) if materias else None,
            "titulo": titulo.text.strip() if titulo else None,
            "camara_origen": cam_origen.text.strip() if cam_origen else None,
            "fecha_ingreso": fecha_origen.text.strip() if fecha_origen else None,
            "ley": ley.text.strip() if ley else None,
            "estado": estado.text.strip() if estado else None
        }

        return pd.DataFrame([data])

    except Exception as e:
        print(f"Error al procesar boletín {boletin}: {e}")
        data = {
            "boletin": boletin_num,
            "materias": [],
            "materias_texto": None,
            "titulo": None,
            "camara_origen": None,
            "fecha_ingreso": None,
            "ley": None,
            "estado": None
        }
        return pd.DataFrame([data])
    
def extraer_materia(materias):
    
    prompt = f"""Eres un analista experto en el Congreso Nacional de Chile. Tu tarea es clasificar un proyecto de ley según sus materias temáticas generales.

Recibirás una lista de materias separadas por punto y coma (;), por ejemplo:

"AGUINALDO SECTOR PÚBLICO; FUNCIONARIOS PÚBLICOS; REAJUSTE DE REMUNERACIONES"

A partir de esa lista, identifica los ámbitos temáticos más generales a los que pertenece el proyecto. 
Utiliza SOLO las siguientes categorías:

1. Educación  
2. Salud  
3. Trabajo y Previsión  
4. Economía y Hacienda  
5. Seguridad y Justicia  
6. Medio Ambiente y Energía  
7. Vivienda y Urbanismo  
8. Gobierno y Política  
9. Relaciones Exteriores  
10. Cultura y Deporte  
11. Transporte y Telecomunicaciones  
12. Derechos Humanos y Género  
13. no cumple (si no corresponde a ninguno de los ámbitos anteriores)

Devuelve tu respuesta **en formato JSON**, siguiendo este esquema:

{{
  "materias_originales": "<texto exacto recibido>",
  "ambitos_detectados": ["<ámbito1>", "<ámbito2>", ...]
}}

Ejemplo de salida esperada:

{{
  "materias_originales": "AGUINALDO SECTOR PÚBLICO; FUNCIONARIOS PÚBLICOS; REAJUSTE DE REMUNERACIONES",
  "ambitos_detectados": ["Trabajo y Previsión", "Economía y Hacienda"]
}}

Si no hay coincidencias evidentes con los ámbitos definidos, asigna ["no cumple"].

Texto:
{materias}
    """
    # Build the message list
    messages = [
        {"role": "user", "content": prompt}
    ]
    # Call the API (stream=False for a quick test, then you can enable streaming)
    response = client.chat(model_name, messages=messages, stream=False)
    json_str = response["message"]["content"].replace("```json", "").replace("```", "").strip()
    try:
        datos = json.loads(json_str)
    except json.JSONDecodeError as e:
        datos = {}
        
    for k, v in list(datos.items()):
        if isinstance(v, str) and v.strip().lower() == "desconocido":
            datos[k] = None
    return datos

In [37]:
#!pip install -q ollama

# Import and create a client that points explicitly to the local daemon
from ollama import Client

# Explicit host helps avoid accidental DNS problems
client = Client(host="http://127.0.0.1:11434")   # <- change if your server runs elsewhere

# Pick a model that you have already pulled locally
model_name = "gpt-oss:120b-cloud"

In [34]:
for periodo in periodos:
    df_det = pd.read_csv(DATA_DIR / periodo / "detalle.csv")
    boletines = (
    df_det.loc[df_det["tipo_iniciativa"] == "Proyecto de ley", "descripcion"]
    .astype(str).str.extract(r"Bolet[ií]n\s*N[°º]?\s*(\d+)")[0]
    .dropna()
    .drop_duplicates()
    )
    df_boletines = pd.concat(
        [get_boletin(b) for b in tqdm(boletines)],
        ignore_index=True
    )
    
    df_boletines["materias norm"] = df_boletines["materias"].apply(extraer_materia)
    df_boletines["ambitos"] = df_boletines["materias norm"].apply(
        lambda x: x.get("ambitos_detectados") if isinstance(x, dict) else ["no cumple"]
    )
    df_boletines.drop(columns=["materias", "materias_texto", "materias norm"], inplace=True)
    df_boletines.to_csv(DATA_DIR / periodo / "boletines.csv", index=False, encoding="utf-8", lineterminator="\n")

100%|████████████████████████████████████████████████████████████████████████████████| 656/656 [08:31<00:00,  1.28it/s]
